In [1]:
%cd ~/work/deeplearning/

/home/asaeed9/work/deeplearning


In [3]:
from theano.sandbox import cuda
cuda.use('gpu1')

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

WARNING (theano.sandbox.cuda): Ignoring call to use(1), GPU number 0 is already in use.


In [4]:
%matplotlib inline
from __future__ import print_function, division
#path="../data/2cat/sample"
import utils; reload(utils)
from utils import *
from IPython.display import FileLink
from keras.preprocessing import image, sequence
import os, sys
from shutil import copyfile

####
from keras.layers.convolutional import *
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.optimizers import SGD, RMSprop, Adam

Using Theano backend.


In [5]:
batch_size=64

In [6]:
path="/home/asaeed9/work/data/2cat/sample/"
results_path = "/home/asaeed9/work/data/2cat/sample/results"
test_path = path + '/test/' #We use all the test data

In [7]:
%cd ../data/2cat/train

/home/asaeed9/work/data/2cat/train


In [8]:
#copy training images
def copy_samples(n):
    g = glob('*.jpg')
    shuf = np.random.permutation(g)
    for i in range(n): copyfile(shuf[i], '../sample/train/' + shuf[i]) 
    %mv ../sample/train/cat*.jpg ../sample/train/cats/
    %mv ../sample/train/dog*.jpg ../sample/train/dogs/
    
    %cd ../valid

    g = glob('*.jpg')
    shuf = np.random.permutation(g)
    for i in range(500): copyfile(shuf[i], '../sample/valid/' + shuf[i]) 
    %mv ../sample/valid/cat*.jpg ../sample/valid/cats/
    %mv ../sample/valid/dog*.jpg ../sample/valid/dogs/
    %cd /home/asaeed9/work/data/2cat/train

In [9]:
def clean_up():
    %rm $path/train/cats/*
    %rm $path/train/dogs/*
    %rm $path/valid/cats/*
    %rm $path/valid/dogs/*

In [10]:
clean_up()

In [11]:
%cd /home/asaeed9/work/data/2cat/train
#for n in range(1000, 5001, 1000):
n = 1500
print('sample size: {}'.format(n))
copy_samples(n)

/home/asaeed9/work/data/2cat/train
sample size: 1500
/home/asaeed9/work/data/2cat/valid
/home/asaeed9/work/data/2cat/train


In [12]:
def conv1(batches):
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(3,256,256)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Flatten(),
            Dense(200, activation='relu'),
            BatchNormalization(),
            Dense(2, activation='softmax')
        ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
#     model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
#                      nb_val_samples=val_batches.nb_sample)
    model.optimizer.lr = 0.001
    model.fit_generator(batches, batches.nb_sample, nb_epoch=10, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    return model

In [13]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)

tr_batches = gen_t.flow_from_directory(path + 'train', batch_size=batch_size)
val_batches = gen_t.flow_from_directory(path + 'valid', class_mode='categorical', shuffle=True, batch_size=batch_size * 2)

Found 1500 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [14]:
conv1(tr_batches)

1 #include <Python.h>
2 #include <iostream>
3 #include "theano_mod_helper.h"
4 #include "cudnn.h"
5 //////////////////////
6 ////  Support Code
7 //////////////////////
8 
9 #if PY_MAJOR_VERSION >= 3
10 #define PyInt_FromLong PyLong_FromLong
11 #endif
12 
13 
14     namespace {
15     struct __struct_compiled_op_3442132e74e76f61361fb1c73112eef4 {
16         PyObject* __ERROR;
17 
18         PyObject* storage_V1;
19         
20 
21         __struct_compiled_op_3442132e74e76f61361fb1c73112eef4() {
22             // This is only somewhat safe because we:
23             //  1) Are not a virtual class
24             //  2) Do not use any virtual classes in the members
25             //  3) Deal with mostly POD and pointers
26 
27             // If this changes, we would have to revise this, but for
28             // now I am tired of chasing segfaults because
29             // initialization code had an error and some pointer has
30             // a junk value.
31             memset(this, 0

ERROR (theano.gof.opt): TRACEBACK:
ERROR (theano.gof.opt): Traceback (most recent call last):
  File "/home/asaeed9/anaconda2/lib/python2.7/site-packages/theano/gof/opt.py", line 1982, in process_node
    replacements = lopt.transform(node)
  File "/home/asaeed9/anaconda2/lib/python2.7/site-packages/theano/gof/opt.py", line 1335, in transform
    new_repl = opt.transform(node)
  File "/home/asaeed9/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/dnn.py", line 3793, in local_abstract_batch_norm_train_grad_cudnn
    GpuDnnBatchNormGrad(mode, epsilon=eps)(x, dy, scale, x_mean, x_invstd)
  File "/home/asaeed9/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/dnn.py", line 2391, in __init__
    if version() < (5000, 5000):
  File "/home/asaeed9/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py", line 429, in dnn_version
    profile=False)
  File "/home/asaeed9/anaconda2/lib/python2.7/site-packages/theano/compile/function.py", line 326, in function
    o


['nvcc', '-shared', '-O3', '-Xlinker', '-rpath,/usr/local/cuda/lib64', '-arch=sm_52', '-m64', '-Xcompiler', '-fno-math-errno,-Wno-unused-label,-Wno-unused-variable,-Wno-write-strings,-DCUDA_NDARRAY_CUH=c72d035fdf91890f3b36710688069b2e,-DNPY_NO_DEPRECATED_API=NPY_1_7_API_VERSION,-fPIC,-fvisibility=hidden', '-Xlinker', '-rpath,/home/asaeed9/.theano/compiledir_Linux-3.19--generic-x86_64-with-debian-jessie-sid-x86_64-2.7.13-64/cuda_ndarray', '-I/usr/local/cuda/include', '-I/home/asaeed9/anaconda2/lib/python2.7/site-packages/numpy/core/include', '-I/home/asaeed9/anaconda2/include/python2.7', '-I/home/asaeed9/anaconda2/lib/python2.7/site-packages/theano/gof', '-I/home/asaeed9/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda', '-L/home/asaeed9/anaconda2/lib', '-o', '/home/asaeed9/.theano/compiledir_Linux-3.19--generic-x86_64-with-debian-jessie-sid-x86_64-2.7.13-64/tmpjObHeS/3442132e74e76f61361fb1c73112eef4.so', 'mod.cu', '-lcudnn', '-lpython2.7', '-lcudart']


1 #include <Python.h>
2 #include <iostream>
3 #include "theano_mod_helper.h"
4 #include "cudnn.h"
5 //////////////////////
6 ////  Support Code
7 //////////////////////
8 
9 #if PY_MAJOR_VERSION >= 3
10 #define PyInt_FromLong PyLong_FromLong
11 #endif
12 
13 
14     namespace {
15     struct __struct_compiled_op_3442132e74e76f61361fb1c73112eef4 {
16         PyObject* __ERROR;
17 
18         PyObject* storage_V1;
19         
20 
21         __struct_compiled_op_3442132e74e76f61361fb1c73112eef4() {
22             // This is only somewhat safe because we:
23             //  1) Are not a virtual class
24             //  2) Do not use any virtual classes in the members
25             //  3) Deal with mostly POD and pointers
26 
27             // If this changes, we would have to revise this, but for
28             // now I am tired of chasing segfaults because
29             // initialization code had an error and some pointer has
30             // a junk value.
31             memset(this, 0

ERROR (theano.gof.opt): TRACEBACK:
ERROR (theano.gof.opt): Traceback (most recent call last):
  File "/home/asaeed9/anaconda2/lib/python2.7/site-packages/theano/gof/opt.py", line 1982, in process_node
    replacements = lopt.transform(node)
  File "/home/asaeed9/anaconda2/lib/python2.7/site-packages/theano/gof/opt.py", line 1335, in transform
    new_repl = opt.transform(node)
  File "/home/asaeed9/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/dnn.py", line 3564, in local_abstractconv_cudnn
    conv_mode=conv_mode)
  File "/home/asaeed9/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/dnn.py", line 1369, in dnn_gradinput
    return GpuDnnConvGradI()(kerns, topgrad, out, desc)
  File "/home/asaeed9/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/dnn.py", line 859, in __init__
    if version() < (5000, 5000):
  File "/home/asaeed9/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py", line 429, in dnn_version
    profile=False)
  File "/hom

...

1 #include <Python.h>
2 #include <iostream>
3 #include "theano_mod_helper.h"
4 #include "cudnn.h"
5 //////////////////////
6 ////  Support Code
7 //////////////////////
8 
9 #if PY_MAJOR_VERSION >= 3
10 #define PyInt_FromLong PyLong_FromLong
11 #endif
12 
13 
14     namespace {
15     struct __struct_compiled_op_3442132e74e76f61361fb1c73112eef4 {
16         PyObject* __ERROR;
17 
18         PyObject* storage_V1;
19         
20 
21         __struct_compiled_op_3442132e74e76f61361fb1c73112eef4() {
22             // This is only somewhat safe because we:
23             //  1) Are not a virtual class
24             //  2) Do not use any virtual classes in the members
25             //  3) Deal with mostly POD and pointers
26 
27             // If this changes, we would have to revise this, but for
28             // now I am tired of chasing segfaults because
29             // initialization code had an error and some pointer has
30             // a junk value.
31             memset(this, 0

ERROR (theano.gof.opt): Traceback (most recent call last):
  File "/home/asaeed9/anaconda2/lib/python2.7/site-packages/theano/gof/opt.py", line 1982, in process_node
    replacements = lopt.transform(node)
  File "/home/asaeed9/anaconda2/lib/python2.7/site-packages/theano/gof/opt.py", line 1335, in transform
    new_repl = opt.transform(node)
  File "/home/asaeed9/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/dnn.py", line 3856, in local_abstract_batch_norm_inference_cudnn
    mode, eps)
  File "/home/asaeed9/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/dnn.py", line 3139, in dnn_batch_normalization_test
    batchnorm_op = GpuDnnBatchNormInference(mode=mode, epsilon=epsilon)
  File "/home/asaeed9/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/dnn.py", line 2460, in __init__
    super(GpuDnnBatchNormInference, self).__init__(mode=mode, epsilon=epsilon)
  File "/home/asaeed9/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/dnn.py", line 2391, 


['nvcc', '-shared', '-O3', '-Xlinker', '-rpath,/usr/local/cuda/lib64', '-arch=sm_52', '-m64', '-Xcompiler', '-fno-math-errno,-Wno-unused-label,-Wno-unused-variable,-Wno-write-strings,-DCUDA_NDARRAY_CUH=c72d035fdf91890f3b36710688069b2e,-DNPY_NO_DEPRECATED_API=NPY_1_7_API_VERSION,-fPIC,-fvisibility=hidden', '-Xlinker', '-rpath,/home/asaeed9/.theano/compiledir_Linux-3.19--generic-x86_64-with-debian-jessie-sid-x86_64-2.7.13-64/cuda_ndarray', '-I/usr/local/cuda/include', '-I/home/asaeed9/anaconda2/lib/python2.7/site-packages/numpy/core/include', '-I/home/asaeed9/anaconda2/include/python2.7', '-I/home/asaeed9/anaconda2/lib/python2.7/site-packages/theano/gof', '-I/home/asaeed9/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda', '-L/home/asaeed9/anaconda2/lib', '-o', '/home/asaeed9/.theano/compiledir_Linux-3.19--generic-x86_64-with-debian-jessie-sid-x86_64-2.7.13-64/tmpf9MImS/3442132e74e76f61361fb1c73112eef4.so', 'mod.cu', '-lcudnn', '-lpython2.7', '-lcudart']


ERROR (theano.gof.opt): Optimization failure due to: local_abstractconv_check
ERROR (theano.gof.opt): node: AbstractConv2d{convdim=2, border_mode='valid', subsample=(1, 1), filter_flip=True, imshp=(None, None, None, None), kshp=(32, 3, 3, 3), filter_dilation=(1, 1)}(if{gpu}.0, convolution2d_1_W)
ERROR (theano.gof.opt): TRACEBACK:
ERROR (theano.gof.opt): Traceback (most recent call last):
  File "/home/asaeed9/anaconda2/lib/python2.7/site-packages/theano/gof/opt.py", line 1982, in process_node
    replacements = lopt.transform(node)
  File "/home/asaeed9/anaconda2/lib/python2.7/site-packages/theano/tensor/nnet/opt.py", line 615, in local_abstractconv_check
    node.op.__class__.__name__)
AssertionError: AbstractConv2d Theano optimization failed: there is no implementation available supporting the requested options. Did you exclude both "conv_dnn" and "conv_gemm" from the optimizer? If on GPU, is cuDNN available and does the GPU support it? If on CPU, do you have a BLAS library installed

AssertionError: AbstractConv2d Theano optimization failed: there is no implementation available supporting the requested options. Did you exclude both "conv_dnn" and "conv_gemm" from the optimizer? If on GPU, is cuDNN available and does the GPU support it? If on CPU, do you have a BLAS library installed Theano can link against?

In [ ]:
results_path

In [ ]:
mod.save_weights(results_path+ '/' +'ft_15000.e10')